# Imports

In [1]:
from utils import *
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import pickle
from datetime import datetime as dt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import keras_tuner
import warnings
import calendar
import collections.abc
collections.Iterable = collections.abc.Iterable
warnings.filterwarnings("ignore")

2023-04-21 17:09:37.980391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 17:09:38.576436: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-21 17:09:38.576507: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-21 17:09:38.576518: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [ ]:
print(tf.config.list_physical_devices('GPU'))

# Data Ingestion and Preparation

In [ ]:
X = train
Y = train.shift(-1, fill_value=0)
X_train, X_test, y_train, y_test, X_test_copy, X_test_index = data_prep(X, Y)

# RNN model

In [ ]:
rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(100, return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.SimpleRNN(100, return_sequences=True),
    keras.layers.SimpleRNN(100, return_sequences=True),
    keras.layers.Dense(1, activation='relu')
])
pred_rnn = predict(rnn, X_train, y_train, X_test, X_test_copy, X_test_index)

## RNN Submission

In [ ]:
Y = pd.DataFrame()
for feature in train.columns:
    for step in range(1, 17):
        Y[f'{feature}_step_ahead_{step}'] = train[feature].shift(-step, fill_value=0)
Y

In [ ]:
X = train
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=5, shuffle=False)
X_test_copy = X_test
X_test_index = X_test.index
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)
y_train = y_train.values.reshape(y_train.shape[0], y_train.shape[1], 16)
y_test = y_test.values.reshape(y_test.shape[0], y_test.shape[1], 16)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(100, return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.SimpleRNN(100, return_sequences=True),
    keras.layers.SimpleRNN(100, return_sequences=True),
    keras.layers.Dense(16)
])
rnn.compile(loss=keras.losses.MeanSquaredLogarithmicError(), optimizer=tf.keras.optimizers.Adam())
rnn.summary()
rnn.fit(X_train, y_train, epochs=10, callbacks=[es])
pred_rnn = abs(rnn.predict(X_test))

In [ ]:
pred = pd.DataFrame(pred_rnn[168].T, columns=X.columns, index=test.index.unique())
pred

In [ ]:
id = []
sales = []
for i in test.id:
    row = test[test['id'] == i]
    column = f"{row['store_nbr'][0]}_{row['family'][0].replace('/', '_')}"
    value = pred[column][pred.index == row.index[0]][0]
    # The info from the ID row is taken and used to form a variable called column. 
    # This column variable refers to the correct store in the dataframe and the equality check on the indexes refer to the proper dates lining up.
    id.append(i)
    sales.append(value)
rnn_submission = pd.DataFrame({'id': id, 'sales': sales})
rnn_submission

In [ ]:
rnn_submission.to_csv('RNN_submission.csv', header=True, index=False)

# LSTM

In [ ]:
X = train
Y = train.shift(-1, fill_value=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=5, shuffle=False)
X_test_copy = X_test
X_test_index = X_test.index
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)
y_train = y_train.values.reshape(y_train.shape[0], 1782, 1)
y_test = y_test.values.reshape(y_test.shape[0], 1782, 1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
lstm = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.Dense(1)
])
lstm.compile(loss=keras.losses.MeanSquaredLogarithmicError(), optimizer=tf.keras.optimizers.Adam())
lstm.summary()
lstm.fit(X_train, y_train, epochs=100, callbacks=[es])
pred_lstm = abs(lstm.predict(X_test))

In [ ]:
print(pred_lstm.shape)
pred_lstm

In [ ]:
pred_lstm = pred_lstm.reshape(169, 1782)
pred = pd.DataFrame(pred_lstm, columns=X.columns, index=X_test_index)
pred

In [ ]:
compare('1_AUTOMOTIVE')

In [ ]:
compare('1_BABY CARE')

In [ ]:
compare('1_BEAUTY')

In [ ]:
compare('1_BEVERAGES')

In [ ]:
compare('1_BOOKS')

In [ ]:
compare('1_BREAD_BAKERY')

In [ ]:
compare('1_CELEBRATION')

In [ ]:
compare('1_CLEANING')

In [ ]:
compare('1_DAIRY')

In [ ]:
compare('1_DELI')

In [ ]:
compare('1_EGGS')

In [ ]:
compare('1_FROZEN FOODS')

In [ ]:
compare('1_GROCERY I')

In [ ]:
compare('1_GROCERY II')

In [ ]:
compare('1_HARDWARE')

In [ ]:
compare('1_HOME AND KITCHEN I')

In [ ]:
compare('1_HOME AND KITCHEN II')

In [ ]:
compare('1_HOME APPLIANCES')

In [ ]:
compare('1_HOME CARE')

In [ ]:
compare('1_LADIESWEAR')

In [ ]:
compare('1_LAWN AND GARDEN')

In [ ]:
compare('1_LINGERIE')

In [ ]:
compare('1_LIQUOR,WINE,BEER')

In [ ]:
pred

In [ ]:
col = list(zip([i+1 for i in range(len(train.columns))], train.columns))
col

## LSTM Submission

In [ ]:
Y = pd.DataFrame()
for feature in train.columns:
    for step in range(1, 17):
        Y[f'{feature}_step_ahead_{step}'] = train[feature].shift(-step, fill_value=0)
Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=5, shuffle=False)
X_test_copy = X_test
X_test_index = X_test.index
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)
y_train = y_train.values.reshape(y_train.shape[0], 1782, 16)
y_test = y_test.values.reshape(y_test.shape[0], 1782, 16)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
lstm = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.Dense(16)
])
lstm.compile(loss=keras.losses.MeanSquaredLogarithmicError(), optimizer=tf.keras.optimizers.Adam())
lstm.summary()
lstm.fit(X_train, y_train, epochs=100, callbacks=[es])
pred_lstm = abs(lstm.predict(X_test))

In [ ]:
pred = pd.DataFrame(pred_lstm[168].T, columns=X.columns, index=test.index.unique())
pred

In [ ]:
id = []
sales = []
for i in test.id:
    row = test[test['id'] == i]
    column = f"{row['store_nbr'][0]}_{row['family'][0].replace('/', '_')}"
    value = pred[column][pred.index == row.index[0]][0]
    # The info from the ID row is taken and used to form a variable called column. 
    # This column variable refers to the correct store in the dataframe and the equality check on the indexes refer to the proper dates lining up.
    id.append(i)
    sales.append(value)
lstm_submission = pd.DataFrame({'id': id, 'sales': sales})
lstm_submission

In [ ]:
lstm_submission.to_csv('LSTM_submission.csv', header=True, index=False)

# Further Experimentation

## Exog as X

In [ ]:
X = exog
Y = train
X_train, X_test, y_train, y_test, X_test_copy, X_test_index = data_prep(X, Y)

In [ ]:
%load_ext tensorboard


In [ ]:
%tensorboard --logdir logs

In [ ]:
log_dir = "logs/fit/" + dt.now().strftime('%m/%d/%Y %-I:%M:%S %p')
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
lstm = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences=True, input_shape=[None, X_train.shape[2]]),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.Dense(1, activation='relu')
])
pred = predict(lstm, X_train, y_train, X_test, X_test_copy, X_test_index)

In [ ]:
print(pred_lstm.shape)
pred_lstm

In [ ]:
pred_lstm = pred_lstm.reshape(169, 1782)
pred = pd.DataFrame(pred_lstm, columns=Y.columns, index=y_test_index)
pred

In [ ]:
compare('1_AUTOMOTIVE')

In [ ]:
compare('1_CLEANING')

In [ ]:
compare('1_DAIRY')

In [ ]:
compare('1_DELI')

## Exog concatenated with train as X

### One LSTM cell for each time series

In [ ]:
X = pd.concat([exog, train], axis=1)
Y = X.shift(-1, fill_value=0)
X_train, X_test, y_train, y_test , X_test_copy, X_test_index = data_prep(X, Y)

In [ ]:
lstm = keras.models.Sequential([
    keras.layers.LSTM(X.shape[1], return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.Dense(1, activation='relu')
])
lstm.compile(loss=keras.losses.MeanSquaredLogarithmicError(), optimizer=tf.keras.optimizers.Adam())
lstm.summary()
pred = predict(lstm, X_train, y_train, X_test, X_test_index)

In [ ]:
compare(X_test_copy.iloc[:, -1782:],pred)

### Triple layer of 100 cells

In [ ]:
exog = pickle.load(open('exog.pkl', 'rb'))
X = pd.concat([exog, train], axis=1)
Y = X.shift(-1, fill_value=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=5, shuffle=False)
X_test_copy = X_test
X_test_index = X_test.index
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)
y_train = y_train.values.reshape(y_train.shape[0], y_train.shape[1], 1)
y_test = y_test.values.reshape(y_test.shape[0], y_test.shape[1], 1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
lstm = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.Dense(1, activation='relu')
])
lstm.compile(loss=keras.losses.MeanSquaredLogarithmicError(), optimizer=tf.keras.optimizers.Adam())
lstm.summary()
lstm.fit(X_train, y_train, epochs=200, callbacks=[es])
pred_lstm = lstm.predict(X_test)

In [ ]:
print(pred_lstm.shape)
pred_lstm

In [ ]:
pred_lstm = pred_lstm.reshape(X_test.shape[0], X_test.shape[1])
pred = pd.DataFrame(pred_lstm, columns=X.columns, index=X_test_index)
pred = pred.iloc[:, -1782:]
pred

In [ ]:
compare('1_AUTOMOTIVE')

In [ ]:
compare('1_BABY CARE')

In [ ]:
compare('1_BEAUTY')

In [ ]:
compare('1_BEVERAGES')

In [ ]:
compare('1_BOOKS')

In [ ]:
compare('1_BREAD_BAKERY')

In [ ]:
compare('1_CELEBRATION')

In [ ]:
compare('1_CLEANING')

In [ ]:
compare('1_DAIRY')

In [ ]:
compare('1_DELI')

In [ ]:
compare('1_EGGS')

In [ ]:
compare('1_FROZEN FOODS')

In [ ]:
compare('1_GROCERY I')

In [ ]:
compare('1_GROCERY II')

In [ ]:
compare('1_HARDWARE')

In [ ]:
compare('1_HOME AND KITCHEN I')

In [ ]:
compare('1_HOME AND KITCHEN II')

In [ ]:
compare('1_HOME APPLIANCES')

In [ ]:
compare('1_HOME CARE')

In [ ]:
compare('1_LADIESWEAR')

In [ ]:
compare('1_LAWN AND GARDEN')

In [ ]:
compare('1_LINGERIE')

In [ ]:
compare('1_LIQUOR,WINE,BEER')

# Another Submission

In [3]:
exog = pickle.load(
    open('/home/nasibul/Desktop/e-commerce/Advanced ARIMA/Data/exog.pkl', 'rb'))
X = pd.concat([exog, train], axis=1)
Y = pd.DataFrame()
for feature in X.columns:
    for step in range(1, 17):
        Y[f'{feature}_step_ahead_{step}'] = X[feature].shift(-step, fill_value=0)
Y

,dcoilwtico_step_ahead_1,dcoilwtico_step_ahead_2,dcoilwtico_step_ahead_3,dcoilwtico_step_ahead_4,dcoilwtico_step_ahead_5,dcoilwtico_step_ahead_6,dcoilwtico_step_ahead_7,dcoilwtico_step_ahead_8,dcoilwtico_step_ahead_9,dcoilwtico_step_ahead_10,...,54_SEAFOOD_step_ahead_7,54_SEAFOOD_step_ahead_8,54_SEAFOOD_step_ahead_9,54_SEAFOOD_step_ahead_10,54_SEAFOOD_step_ahead_11,54_SEAFOOD_step_ahead_12,54_SEAFOOD_step_ahead_13,54_SEAFOOD_step_ahead_14,54_SEAFOOD_step_ahead_15,54_SEAFOOD_step_ahead_16
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,93.14,92.97,93.12,93.12,93.20,93.20,93.21,93.08,93.81,93.60,...,1.0,1.0,1.0,0.0,1.0,5.0,0.0,2.0,1.0,2.0
2013-01-02,92.97,93.12,93.12,93.20,93.20,93.21,93.08,93.81,93.60,93.60,...,1.0,1.0,0.0,1.0,5.0,0.0,2.0,1.0,2.0,0.0
2013-01-03,93.12,93.12,93.20,93.20,93.21,93.08,93.81,93.60,93.60,94.27,...,1.0,0.0,1.0,5.0,0.0,2.0,1.0,2.0,0.0,1.0
2013-01-04,93.12,93.20,93.20,93.21,93.08,93.81,93.60,93.60,94.27,94.27,...,0.0,1.0,5.0,0.0,2.0,1.0,2.0,0.0,1.0,7.0
2013-01-05,93.20,93.20,93.21,93.08,93.81,93.60,93.60,94.27,94.27,93.26,...,1.0,5.0,0.0,2.0,1.0,2.0,0.0,1.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,48.81,47.59,47.59,47.57,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-12,47.59,47.59,47.57,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-13,47.59,47.57,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=5, shuffle=False)
X_test_copy = X_test
X_test_index = X_test.index
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)
y_train = y_train.values.reshape(y_train.shape[0], X.shape[1], 16)
y_test = y_test.values.reshape(y_test.shape[0], X.shape[1], 16)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1515, 1808, 1)
(169, 1808, 1)
(1515, 1808, 16)
(169, 1808, 16)


In [ ]:
lstm = keras.models.Sequential([
    keras.layers.LSTM(X.shape[1], return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.Dense(16, activation='relu')
])
lstm.compile(loss=keras.losses.MeanSquaredLogarithmicError(), optimizer=tf.keras.optimizers.Adam())
lstm.summary()
lstm.fit(X_train, y_train, epochs=200, callbacks=[es])
pred_lstm = lstm.predict(X_test)

In [ ]:
pred = pd.DataFrame(pred_lstm[168].T, columns=X.columns, index=test.index.unique())
pred

In [ ]:
pred = pred.iloc[:, -1782:]
pred

In [ ]:
id = []
sales = []
for i in test.id:
    row = test[test['id'] == i]
    column = f"{row['store_nbr'][0]}_{row['family'][0].replace('/', '_')}"
    value = pred[column][pred.index == row.index[0]][0]
    # The info from the ID row is taken and used to form a variable called column. 
    # This column variable refers to the correct store in the dataframe and the equality check on the indexes refer to the proper dates lining up.
    id.append(i)
    sales.append(value)
lstm_submission = pd.DataFrame({'id': id, 'sales': sales})
lstm_submission